In [5]:
import os
from pathlib import Path

import pandas as pd
import numpy as np


from tqdm.notebook import tqdm

In [6]:
def calculate_euclidean_distance(row):
    return np.sqrt(
        (row["gaze_angle_x"] - row["target_angle_x"]) ** 2
        + (row["gaze_angle_y"] - row["target_angle_y"]) ** 2
    )


def calculate_accuracy_and_std(df):
    eye_tracker = df["eye_tracker"].iloc[0]
    
    accuracy_results = []
    std_results = []

    for trial in df["trial_number"].unique():
        trial_data = df[df["trial_number"] == trial]

        if eye_tracker == 'EyeLink 1000 Plus':
            errors = trial_data.apply(calculate_euclidean_distance, axis=1)
            accuracy = errors.mean()
            std = errors.std()
        else: 
            accuracy = trial_data['gaze_target_angle'].mean()
            std = trial_data['gaze_target_angle'].std()
        n_samples = len(trial_data)

        # Append to accuracy DataFrame
        accuracy_results.append({
            "eye_tracker": trial_data["eye_tracker"].iloc[0],
            "participant_id": trial_data["participant_id"].iloc[0],
            "trial_number": trial,
            "trial_condition": trial_data["trial_condition"].iloc[0],
            "accuracy": accuracy,
            "n_samples": n_samples,
        })

        # Append to std DataFrame
        std_results.append({
            "eye_tracker": df["eye_tracker"].iloc[0],
            "participant_id": df["participant_id"].iloc[0],
            "trial_number": trial,
            "trial_condition":trial_data["trial_condition"].iloc[0],
            "std": std,
            "n_samples": n_samples,
        })

    accuracy_df = pd.DataFrame(accuracy_results)
    std_df = pd.DataFrame(std_results)

    return accuracy_df, std_df


def calculate_apparent_gaze_shift(df):
    eye_tracker = df["eye_tracker"].iloc[0]

    results = []

    # Get all trials
    trials = sorted(df["trial_number"].unique())

    # Process pairs of trials (odd-even pairs)
    for i in range(0, len(trials) - 1, 2):
        dilated_trial = trials[i]  
        constricted_trial = trials[i + 1] 

        # Get data for each trial
        constricted_data = df[df["trial_number"] == constricted_trial]
        dilated_data = df[df["trial_number"] == dilated_trial]

        if eye_tracker == 'EyeLink 1000 Plus':
            # Calculate mean gaze positions
            constricted_x = constricted_data["gaze_angle_x"].mean()
            constricted_y = constricted_data["gaze_angle_y"].mean()
            dilated_x = dilated_data["gaze_angle_x"].mean()
            dilated_y = dilated_data["gaze_angle_y"].mean()
    
            # Calculate distance
            x_diff = dilated_x - constricted_x
            y_diff = dilated_y - constricted_y
            distance = np.sqrt(x_diff**2 + y_diff**2)
        else:
            # Calculate absolute difference in mean angles
            distance = abs(constricted_data['gaze_target_angle'].mean() - dilated_data['gaze_target_angle'].mean())
        results.append(
            {
                "eye_tracker": df["eye_tracker"].iloc[0],
                "participant_id": df["participant_id"].iloc[0],
                "dilated_trial": dilated_trial,
                "constricted_trial": constricted_trial,
                "apparent_gaze_shift": distance,
                "constricted_samples": len(constricted_data),
                "dilated_samples": len(dilated_data),
            }
        )

    # Convert to DataFrame
    results_df = pd.DataFrame(results)

    return results_df


def calculate_rms_s2s(df):
    eye_tracker = df["eye_tracker"].iloc[0]

    results = []

    for trial in df["trial_number"].unique():
        trial_data = df[df["trial_number"] == trial]

        if eye_tracker == 'EyeLink 1000 Plus':
            # Compute consecutive gaze shifts
            dx = np.diff(trial_data['gaze_angle_x'])
            dy = np.diff(trial_data['gaze_angle_y'])
            distances = np.sqrt(dx**2 + dy**2)
        else:
            distances = np.diff(trial_data['gaze_target_angle'])
        rms_s2s_degrees = np.sqrt(np.mean(distances ** 2))

        results.append({
            "eye_tracker": trial_data["eye_tracker"].iloc[0],
            "participant_id": trial_data["participant_id"].iloc[0],
            "trial_number": trial,
            "trial_condition": trial_data["trial_condition"].iloc[0],
            "rms_s2s": rms_s2s_degrees,
            "n_samples": len(trial_data)
        })

    return pd.DataFrame(results)


In [8]:
project_dir_path = Path(r"/Users/salari/Dropbox/DC3/Dev/psa_data_quality")
dataset_dir_path = project_dir_path / "data"

# Get all eye trackers data directories
eye_trackers = ["EyeLink 1000 Plus", "Pupil Core", "SMI ETG", "Pupil Neon", "Tobii Glasses 2"]
data_paths = []
for participant_dir in dataset_dir_path.iterdir():
    if participant_dir.is_dir():
        for eye_tracker in eye_trackers:
            
            data_path = participant_dir / eye_tracker / "data.csv"
            if data_path.exists():
                data_paths.append(data_path)

In [9]:
accuracy_list = []
std_list = []
rms_list = []
apparent_gaze_shift_list = []

for data_path in tqdm(data_paths, desc="Processing", unit="recording"):

    df = pd.read_csv(data_path)
    acc_df, std_df = calculate_accuracy_and_std(df)
    accuracy_list.append(acc_df)
    std_list.append(std_df)

    rms_list.append(calculate_rms_s2s(df))
    apparent_gaze_shift_list.append(calculate_apparent_gaze_shift(df))



# Combine all results into final DataFrames
accuracy_df = pd.concat(accuracy_list, ignore_index=True)
std_df = pd.concat(std_list, ignore_index=True)
rms_df = pd.concat(rms_list, ignore_index=True)
apparent_gaze_shift_df = pd.concat(apparent_gaze_shift_list, ignore_index=True)

# Sort
accuracy_df = accuracy_df.sort_values(by=["eye_tracker", "participant_id", "trial_number"])
std_df = std_df.sort_values(by=["eye_tracker", "participant_id", "trial_number"])
rms_df = rms_df.sort_values(by=["eye_tracker", "participant_id", "trial_number"])
apparent_gaze_shift_df = apparent_gaze_shift_df.sort_values(by=["eye_tracker", "participant_id"])  


Processing:   0%|          | 0/102 [00:00<?, ?recording/s]

In [27]:
# Define output directory
output_dir = project_dir_path / "quality_metrics"
output_dir.mkdir(exist_ok=True)  # Create the directory if it doesn't exist

# Save each DataFrame to a CSV file
accuracy_df.to_csv(output_dir / "accuracy.csv", index=False)
std_df.to_csv(output_dir / "std.csv", index=False)
rms_df.to_csv(output_dir / "rms_s2s.csv", index=False)
apparent_gaze_shift_df.to_csv(output_dir / "apparent_gaze_shift.csv", index=False)

print(f"Saved CSV files to: {output_dir}")

Saved CSV files to: C:\Users\wksadmin\Dropbox\DC3\Dev\psa_data_quality\quality_metrics


In [11]:
# Assuming your DataFrame is named df
accuracy_df[(accuracy_df['eye_tracker'] == 'Pupil Core') & (accuracy_df['participant_id'] == 141)]


,eye_tracker,participant_id,trial_number,trial_condition,accuracy,n_samples
540,Pupil Core,141,1,dilated,1.512138,453
543,Pupil Core,141,2,constricted,2.011453,898
541,Pupil Core,141,3,dilated,2.297144,786
544,Pupil Core,141,4,constricted,1.439566,827
542,Pupil Core,141,5,dilated,2.265954,910
545,Pupil Core,141,6,constricted,2.456205,905


In [20]:
df = pd.read_csv(r"C:\Users\wksadmin\Dropbox\DC3\Dev\psa_data_quality\data\141\Pupil Core\data.csv")

In [21]:
constricted_data = df[df["trial_number"] == 2]
dilated_data = df[df["trial_number"] == 1]

In [22]:
constricted_data

,eye_tracker,participant_id,frame,trial_condition,trial_number,target_x,target_y,timestamp,gaze_ori_l_x,gaze_ori_l_y,...,gaze_dir_r_x,gaze_dir_r_y,gaze_dir_r_z,gaze_pos_vid_x,gaze_pos_vid_y,gaze_pos_3d_x,gaze_pos_3d_y,gaze_pos_3d_z,confidence,gaze_target_angle
2286,Pupil Core,141,2710,constricted,2,662,468,95041.231482,-41.813159,16.445845,...,0.038326,0.053705,0.997821,681.044231,447.969079,55.821590,58.863224,919.888967,0.908149,-0.175021
2287,Pupil Core,141,2711,constricted,2,662,468,95045.267982,-41.813159,16.445845,...,0.038158,0.057165,0.997635,680.941323,449.425875,55.475227,60.311421,916.050970,0.934935,-0.170512
2288,Pupil Core,141,2711,constricted,2,662,468,95049.257482,-41.813159,16.445845,...,0.038158,0.057165,0.997635,680.222587,449.390608,56.992682,62.861830,955.485577,0.921254,-0.165989
2289,Pupil Core,141,2711,constricted,2,662,468,95053.216982,-41.813159,16.445845,...,0.038140,0.056422,0.997678,680.215983,449.089373,56.987548,62.500928,955.555294,0.938394,-0.166742
2290,Pupil Core,141,2711,constricted,2,662,468,95057.197982,-41.813159,16.445845,...,0.038140,0.056422,0.997678,679.140796,447.342812,59.332307,64.340518,1018.265614,0.945978,-0.164450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3178,Pupil Core,141,2818,constricted,2,663,470,98649.351982,-41.777723,16.480632,...,0.058006,0.049184,0.997104,688.975890,439.449923,117.364381,88.335694,1659.910490,0.997268,-0.254891
3179,Pupil Core,141,2818,constricted,2,663,470,98653.332982,-41.777723,16.480632,...,0.057801,0.048555,0.997147,688.869605,439.237051,116.541500,87.438667,1651.444263,0.991819,-0.254771
3180,Pupil Core,141,2818,constricted,2,663,470,98657.300482,-41.777723,16.480632,...,0.057801,0.048555,0.997147,688.830276,438.309180,116.597330,85.602059,1653.497116,0.989466,-0.257055
3181,Pupil Core,141,2819,constricted,2,663,470,98661.245482,-41.777723,16.480632,...,0.057801,0.048555,0.997147,688.633362,437.014051,118.086441,84.249889,1680.697237,0.988747,-0.259304


In [23]:
dilated_data

,eye_tracker,participant_id,frame,trial_condition,trial_number,target_x,target_y,timestamp,gaze_ori_l_x,gaze_ori_l_y,...,gaze_dir_r_x,gaze_dir_r_y,gaze_dir_r_z,gaze_pos_vid_x,gaze_pos_vid_y,gaze_pos_3d_x,gaze_pos_3d_y,gaze_pos_3d_z,confidence,gaze_target_angle
0,Pupil Core,141,1992,dilated,1,656,480,70951.197482,-40.562548,15.781570,...,-0.005557,0.071655,0.997414,650.276544,460.414583,15.772216,57.650593,723.393296,1.000000,0.002059
1,Pupil Core,141,1992,dilated,1,656,480,70955.164982,-40.562548,15.781570,...,-0.004666,0.072290,0.997373,650.756552,460.477168,16.391960,58.353050,731.468462,1.000000,-0.001414
2,Pupil Core,141,1992,dilated,1,656,480,70959.226982,-40.562548,15.781570,...,-0.004666,0.072290,0.997373,650.406264,461.311311,16.349936,60.160367,744.234874,1.000000,0.002958
3,Pupil Core,141,1992,dilated,1,656,480,70963.352982,-40.562548,15.781570,...,-0.002463,0.071840,0.997413,651.596090,460.635781,17.981799,61.274645,766.137653,1.000000,-0.007385
4,Pupil Core,141,1992,dilated,1,656,480,70967.310482,-40.562548,15.781570,...,-0.002463,0.071840,0.997413,651.470073,459.967586,17.958820,60.963339,770.431575,1.000000,-0.007835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580,Pupil Core,141,2061,dilated,1,658,479,73291.337482,-40.571851,15.844299,...,-0.019112,0.052758,0.998424,640.350221,447.246138,6.340025,43.128878,684.917330,0.816669,0.058852
581,Pupil Core,141,2061,dilated,1,658,479,73293.399482,-60.224923,-155.987977,...,NaN,NaN,NaN,660.315342,441.846901,18.317927,29.868914,531.854050,0.818120,-0.101613
582,Pupil Core,141,2062,dilated,1,656,479,73301.251482,-60.224923,-155.987977,...,NaN,NaN,NaN,686.286808,497.093595,34.709108,65.036627,512.706549,0.915304,-0.186841
583,Pupil Core,141,2062,dilated,1,656,479,73317.225482,-60.224923,-155.987977,...,NaN,NaN,NaN,844.467521,1022.608836,99.435886,294.514173,221.194167,0.990000,-0.385904


In [24]:
# Method 1: Check if any values are negative
has_negative = (df['gaze_target_angle'] < 0).any()
print(f"Has negative values: {has_negative}")

# Method 2: Count how many negative values
negative_count = (df['gaze_target_angle'] < 0).sum()
print(f"Number of negative values: {negative_count}")

# Method 3: See the minimum value (if it's negative, there are negative values)
min_value = df['gaze_target_angle'].min()
print(f"Minimum value: {min_value}")

# Method 4: Filter and view the negative values
negative_values = df[df['gaze_target_angle'] < 0]
print(f"Shape of negative values DataFrame: {negative_values.shape}")

Has negative values: True
Number of negative values: 3052
Minimum value: -0.925904446791418
Shape of negative values DataFrame: (3052, 29)
